# Размещение баннеров

## Постановка задачи
Представим, что международное круизное агентство "Carnival Cruise Line" решило себя разрекламировать с помощью баннеров и обратилось для этого к Вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с Вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры - легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях - Foursqare.
Часть открытых данных есть, например, на сайте archive.org:
https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив с этой страницы.

## Преобразование данных
Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру:

id | user_id | venue_id | latitude | longitude | created_at

---------+---------+----------+-------------------+-------------------+---------------------

984301 | 2041916 | 5222 | | | 2012-04-21 17:39:01

984222 | 15824 | 5222 | 38.8951118 | -77.0363658 | 2012-04-21 17:43:47

984315 | 1764391 | 5222 | | | 2012-04-21 17:37:18

984234 | 44652 | 5222 | 33.800745 | -84.41052 | 2012-04-21 17:43:43

...

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки не содержащие координат - они неинформативны для нас:

id,user_id,venue_id,latitude,longitude,created_at

984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47

984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43

984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22

...

С помощью pandas построим DataFrame и убедимся, что все 396632 строк с координатами считаны успешно.

In [1]:
import pandas as pd

In [2]:
lines = []
data = []

df = pd.read_csv('checkins.csv')

In [3]:
df = df[(df.latitude != '') & (df.longitude != '')]
df.head()

C:\Users\user\Anaconda2\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-21 17:38:18


In [4]:
df.shape

(396634, 6)

# Кластеризация
Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача - хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в [sklearn user guide](http://scikit-learn.org/stable/modules/clustering.html#mean-shift), а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание: на 396634 строках, кластеризация будет работать долго (порядка часа). Для получения корректного ответа достаточно и 100000 (~2 минуты на "среднем" ноутбуке). Быть очень терпеливым не возбраняется - результат от этого только улучшится.

In [5]:
from sklearn.cluster import MeanShift

In [6]:
data = df[['latitude', 'longitude']]
data = data.applymap(float)

In [7]:
data_sample = data.sample(100000)

In [8]:
%%time
clst = MeanShift(bandwidth = 0.1)
clst.fit(data_sample)

Wall time: 2min 56s


Некоторые из получившихся кластеров содержат слишком мало точек - такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте, можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

38.8951118,-77.0363658

33.800745,-84.41052

45.5234515,-122.6762071

...

In [9]:
data_sample['cluster'] = clst.predict(data_sample)

In [10]:
cluster_size = pd.DataFrame(data_sample.pivot_table(index = 'cluster', aggfunc = 'count', values = 'latitude'))
cluster_size.columns = ['clust_size']

In [11]:
cluster_centers_df = pd.DataFrame(clst.cluster_centers_)
cluster_centers_df.columns = ['cent_latitude', 'cent_longitude']

In [12]:
cluster_df = cluster_centers_df.join(cluster_size)
cluster_df.to_csv('clusters.csv', index = None)
cluster_df = cluster_df[cluster_df.clust_size > 15]
cluster_df.head()

,cent_latitude,cent_longitude,clust_size
0,40.717504,-73.989445,14152
1,41.878185,-87.629841,3870
2,33.448489,-112.074085,2809
3,33.447134,-111.903062,2332
4,38.885526,-77.047943,2668


Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу "Carnival Cruise Line" адреса офисов:

 * 33.751277, -118.188740 (_Los Angeles_)
 * 25.867736, -80.324116 (_Miami_)
 * 51.503016, -0.075479 (_London_)
 * 52.378894, 4.885084 (_Amsterdam_)
 * 39.366487, 117.036146 (_Beijing_)
 * -33.868457, 151.205134 (_Sydney_)

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании - широта и долгота этого центра, записанные через пробел.

In [13]:
def get_distance(lat1, lon1, lat2, lon2):
    return ((lat1 - lat2)**2 + (lon1 - lon2)**2) ** 0.5

office_coordinates = [
    (33.751277, -118.188740),
    (25.867736, -80.324116),
    (51.503016, -0.075479),
    (52.378894, 4.885084),
    (39.366487, 117.036146),
    (-33.868457, 151.205134)
]

def get_min_distance_to_office(lat, lon):
    min_dist = None
    for (of_lat, of_lon) in office_coordinates:
        dist = get_distance(lat, lon, of_lat, of_lon)
        if (min_dist is None) or (dist < min_dist):
            min_dist = dist
    return min_dist

In [14]:
cluster_df['min_distance'] = map(get_min_distance_to_office, cluster_df.cent_latitude, cluster_df.cent_longitude)

In [15]:
cluster_df.sort_values('min_distance')[:20]

,cent_latitude,cent_longitude,clust_size,min_distance
276,-33.866233,151.207201,50,0.003036
338,52.372812,4.892475,35,0.009571
55,51.502829,-0.127216,298,0.051737
51,33.812454,-118.142738,270,0.076543
251,25.898195,-80.211893,51,0.116283
24,25.786563,-80.215301,715,0.135756
83,33.896046,-118.063096,146,0.191688
104,26.031767,-80.221162,102,0.193664
35,33.871526,-118.368383,454,0.216175
936,26.122798,-80.391297,20,0.263761
